# 작사가 인공지능 만들기

----------------------------------------
## 평가기준

    1. 가사텍스트 생성 모델이 정상적으로 작동하는가?
      :텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가?
    2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가?
      :특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었는가?
    3. 텍스트 생성모델이 안정적으로 학습되었는가?
       :텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?
       
 --------------------------
 ## Step1. 데이터 다운로드
 
 ## Step2. 데이터 읽어오기

In [1]:
import os, re 
import numpy as np
import tensorflow as tf
import glob

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])


데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?
It goes like this
The fourth, the fifth
The minor fall, the major lift
The baffled king composing Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah Your faith was strong but you needed proof


## Step3. 데이터 정제
    - 문장을 토큰화 했을때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    tmp=preprocess_sentence(sentence)
    if len(tmp.split())>15: continue
    corpus.append(tmp)
    
    
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]


['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

## Step 4. 평가 데이터셋 분리

    - 훈련 데이터와 평가 데이터 분리
    - tokenize() 함수 이용 [데이터 -> Tensor] 변환
    -> sklearn 모듈의 train_test_split() 함수를 이용
        ->훈련 데이터& 평가 데이터를 분리

In [4]:
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,padding='post')
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    4 ...    0    0    0]
 [   2   15 2967 ...    0    0    0]
 [   2   33    7 ...   46    3    0]
 ...
 [   2    4  118 ...    0    0    0]
 [   2  258  194 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f88b8724fa0>


In [5]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 15: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to
11 : it
12 : me
13 : my
14 : in
15 : that


In [6]:
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   50    4   95  303   62   53    9  946 6263    3    0    0    0]
[  50    4   95  303   62   53    9  946 6263    3    0    0    0    0]


In [7]:
from sklearn.model_selection import train_test_split
# (3) train, test 데이터 분리
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                    tgt_input, 
                                                    test_size=0.2, 
                                                    random_state=42)


In [8]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE


VOCAB_SIZE = tokenizer.num_words + 1   


dataset = tf.data.Dataset.from_tensor_slices((enc_train,dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [9]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124810, 14)
Target Train: (124810, 14)


In [10]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
    
embedding_size = 424
hidden_size = 2504# 2의 배수단위로 늘리기 사이즈를 늘리면 에폭도 줄여줘야함
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)
    


# Step 5. 인공지능 만들기
    -  Embedding Size와 Hidden Size를 조절
    -> 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델 설계
        - val_loss값을 생성해야함 
        - val_loss < 2.2

In [11]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')


model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=7, validation_data=(enc_val,dec_val))

Epoch 1/7
487/487 [==============================] - 440s 842ms/step - loss: 3.2474 - val_loss: 2.8776
Epoch 2/7
487/487 [==============================] - 411s 843ms/step - loss: 2.6915 - val_loss: 2.6172
Epoch 3/7
487/487 [==============================] - 412s 845ms/step - loss: 2.3477 - val_loss: 2.4302
Epoch 4/7
487/487 [==============================] - 412s 845ms/step - loss: 2.0060 - val_loss: 2.2896
Epoch 5/7
487/487 [==============================] - 412s 846ms/step - loss: 1.6878 - val_loss: 2.1972
Epoch 6/7
487/487 [==============================] - 412s 847ms/step - loss: 1.4221 - val_loss: 2.1449
Epoch 7/7
487/487 [==============================] - 412s 845ms/step - loss: 1.2226 - val_loss: 2.1348


In [12]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]


    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [13]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i m not gonna crack <end> '

## 회고

    - val_loss
    
    처음 val_loss를 지정하기 위해서 공부를 할때 val_loss를 출력하려면 validation_data=를 model.fit에 추가해 주어야 한다는 것을 알게 되었다. 그런데 어떤 데이터를 넣어주어야 하는지 생각하지 않고 분리전 데이터를 넣었더니 사이즈가 너무 크다는 경고가 나왔다. 같은 조이신 진환님의 도움을 받아서 val_loss 는 val데이터를 넣어 주어야한다는 사실을 알게 되었다. 그리고 이 대화를 통해 내가 train_test_split() 구조에 대해서 착각하고 있다는 사실을 깨닫게 되었다. 그 착각은 enc_train, enc_val를 각각 train과 val 데이터 셋으로 생각하는 것이 아니라 타겟값과 결과 값이라고 착각하고 있었다. 